# Lecture 10
Numerical integration
Some examples illustrating how numerical integration works (using Gauss integration)

Jim Brouzoulis

In [4]:
import numpy as np
import plotly.graph_objects as go

# Functions and exact integrals 
functions = {
    "exp(x)": (lambda x: np.exp(x), np.exp(1) - np.exp(-1)),
    "x^4": (lambda x: x**4, 2/5),
    "cos(x)": (lambda x: np.cos(x), 2*np.sin(1))
}

a, b = -1, 1
x_curve = np.linspace(a, b, 400)

def build_traces(func, exact_val, n):
    # Gauss-Legendre nodes and weights on [-1,1]
    nodes_ref, weights_ref = np.polynomial.legendre.leggauss(n)
    
    # Scale nodes and weights to [a,b]
    nodes = 0.5 * (b - a) * nodes_ref + 0.5 * (b + a)
    weights = 0.5 * (b - a) * weights_ref
    
    func_values = func(nodes)
    approx = np.sum(weights * func_values)
    error = abs((approx - exact_val) / exact_val)

    # Integrand curve
    curve_x = x_curve
    curve_y = func(x_curve)

    # Bars: rectangles for each Gauss point
    bar_x, bar_y = [], []
    for i in range(n):
        half_width = weights[i]
        left = nodes[i] - half_width/2
        right = nodes[i] + half_width/2
        height = func_values[i]
        # Rectangle path
        bar_x += [left, left, right, right, left, None]
        bar_y += [0, height, height, 0, 0, None]

    # Horizontal axis line
    axis_x = [a, b]
    axis_y = [0, 0]

    return (curve_x, curve_y), (bar_x, bar_y), (nodes, [0]*len(nodes)), (axis_x, axis_y), approx, error

def make_slider_steps(func, exact_val, func_name):
    steps = []
    for n in range(1, 11):
        (curve_x, curve_y), (bar_x, bar_y), (points_x, points_y), (axis_x, axis_y), approx_n, error_n = build_traces(func, exact_val, n)
        steps.append(dict(
            method="update",
            args=[{
                "x": [curve_x, bar_x, points_x, axis_x],
                "y": [curve_y, bar_y, points_y, axis_y]
            },
            {"title": f"{func_name}, n={n}, Approx={approx_n:.6f}, Error={error_n:.2e}"}],
            label=str(n)
        ))
    return steps

# Initial function and n
initial_func_name = "exp(x)"
initial_func, initial_exact = functions[initial_func_name]
initial_n = 2

# Build initial traces
(curve_x, curve_y), (bar_x, bar_y), (points_x, points_y), (axis_x, axis_y), approx, error = build_traces(initial_func, initial_exact, initial_n)

fig = go.Figure(data=[
    go.Scatter(x=curve_x, y=curve_y, mode='lines', line=dict(color="#d62728", width=4), name='Integrand'),
    go.Scatter(x=bar_x, y=bar_y, mode='lines', fill='toself', fillcolor='rgba(31,119,180,0.4)', line=dict(color='#1f77b4'), name='Bars'),
    go.Scatter(x=points_x, y=points_y, mode='markers', marker=dict(color='black', size=14), name='Gauss Points'),
    go.Scatter(x=axis_x, y=axis_y, mode='lines', line=dict(color='black', width=4), name='Axis')
])

# Initial slider
sliders = [dict(
    active=initial_n-1,
    currentvalue={"prefix": "Number of Points: "},
    pad={"t": 50},
    steps=make_slider_steps(initial_func, initial_exact, initial_func_name)
)]

# Dropdown for function selection (updates data + slider steps)
dropdown_buttons = []
for func_name, (func, exact_val) in functions.items():
    (curve_x, curve_y), (bar_x, bar_y), (points_x, points_y), (axis_x, axis_y), approx_func, error_func = build_traces(func, exact_val, initial_n)
    dropdown_buttons.append(dict(
        method="update",
        args=[{
            "x": [curve_x, bar_x, points_x, axis_x],
            "y": [curve_y, bar_y, points_y, axis_y]
        },
        {
            "title": f"{func_name}, n={initial_n}, Approx={approx_func:.6f}, Error={error_func:.2e}",
            "sliders": [dict(
                active=initial_n-1,
                currentvalue={"prefix": "Number of Points: "},
                pad={"t": 50},
                steps=make_slider_steps(func, exact_val, func_name)
            )]
        }],
        label=func_name
    ))

fig.update_layout(
    title=f"{initial_func_name}, n={initial_n}, Approx={approx:.6f}, Error={error:.2e}",
    xaxis_title="x",
    yaxis_title="f(x)",
    sliders=sliders,
    updatemenus=[dict(
        buttons=dropdown_buttons,
        direction="down",
        showactive=True,
        x=0.1,
        y=1.15
    )]
)

fig.show()


In [5]:
import numpy as np
import plotly.graph_objects as go

# Integrand
def f_func(x, y):
    return 1 - (0.5*x + 0.2*x**2 + 0.4*y**3)

# Domain
a, b = -1, 1

# Surface grid
N_SURF = 40
x_grid = np.linspace(a, b, N_SURF)
y_grid = np.linspace(a, b, N_SURF)
X_surf, Y_surf = np.meshgrid(x_grid, y_grid)

# Adjustable block transparency
block_opacity = .1

# Build cuboid mesh, Gauss points, and edges
def build_mesh(n, func):
    nodes_ref, weights_ref = np.polynomial.legendre.leggauss(n)
    nodes = 0.5*(b-a)*nodes_ref + 0.5*(b+a)
    weights = 0.5*(b-a)*weights_ref
    xs, ys = nodes, nodes
    heights = np.array([[func(xi, yj) for xi in xs] for yj in ys])
    vx, vy, vz = [], [], []
    i_list, j_list, k_list = [], [], []
    edge_x, edge_y, edge_z = [], [], []
    vertex_count = 0
    for ix in range(n):
        wx = weights[ix]; x0,x1 = xs[ix]-wx/2, xs[ix]+wx/2
        for iy in range(n):
            wy = weights[iy]; y0,y1 = ys[iy]-wy/2, ys[iy]+wy/2
            h = heights[iy, ix]
            base = vertex_count
            verts = [(x0,y0,0),(x1,y0,0),(x1,y1,0),(x0,y1,0),
                     (x0,y0,h),(x1,y0,h),(x1,y1,h),(x0,y1,h)]
            for xx,yy,zz in verts:
                vx.append(xx); vy.append(yy); vz.append(zz)
            faces = [(0,1,2),(0,2,3),(4,5,6),(4,6,7),
                     (0,1,5),(0,5,4),(1,2,6),(1,6,5),
                     (2,3,7),(2,7,6),(3,0,4),(3,4,7)]
            for p,q,r in faces:
                i_list.append(base+p); j_list.append(base+q); k_list.append(base+r)
            edges = [(0,1),(1,2),(2,3),(3,0),(4,5),(5,6),(6,7),(7,4),(0,4),(1,5),(2,6),(3,7)]
            for p,q in edges:
                xs0, ys0, zs0 = verts[p]; xs1, ys1, zs1 = verts[q]
                edge_x += [xs0,xs1,None]; edge_y += [ys0,ys1,None]; edge_z += [zs0,zs1,None]
            vertex_count += 8
    # Gauss points on surface
    x_points = np.repeat(xs, n)
    y_points = np.tile(ys, n)
    z_points = [func(xi, yj) for xi in xs for yj in ys]
    return vx, vy, vz, i_list, j_list, k_list, x_points, y_points, z_points, edge_x, edge_y, edge_z

surface_color = '#d62728'
surface_color = "#e6550d"
# surface_color = "#1f77b4"
# Precompute frames for n = 1..6
n_max = 6
frames = []
for n in range(1, n_max+1):
    vx, vy, vz, i_l, j_l, k_l, x_pts, y_pts, z_pts, ex, ey, ez = build_mesh(n, f_func)
    frames.append(go.Frame(
        name=str(n),
        data=[
            go.Surface(x=X_surf, y=Y_surf, z=f_func(X_surf,Y_surf),
                       surfacecolor=np.ones_like(X_surf),
                    #    colorscale=[[0,'rgb(200,200,200)'],[1,'rgb(200,200,200)']],
                       colorscale=[[0,surface_color],[1,surface_color]],
                       showscale=False, name='Integrand'),
            go.Mesh3d(x=vx, y=vy, z=vz, i=i_l, j=j_l, k=k_l,
                      color='#1f77b4', opacity=block_opacity, name='Blocks'),
            go.Scatter3d(x=x_pts, y=y_pts, z=z_pts,
                         mode='markers', marker=dict(color='black', size=5),
                         name='Gauss points'),
            go.Scatter3d(x=ex, y=ey, z=ez,
                         mode='lines', line=dict(color='black', width=1),
                         name='Edges', showlegend=False)
        ],
        layout=go.Layout(title_text=f'exact integral = {56/15:.4f}, n={n}')
    ))

# Initial state (n=2)
init_n = 2
vx, vy, vz, i_l, j_l, k_l, x_pts, y_pts, z_pts, ex, ey, ez = build_mesh(init_n, f_func)
fig = go.Figure(
    data=[
        go.Surface(x=X_surf, y=Y_surf, z=f_func(X_surf,Y_surf),
                   surfacecolor=np.ones_like(X_surf),
                #    colorscale=[[0,'rgb(200,200,200)'],[1,'rgb(200,200,200)']],
                   colorscale=[[0,surface_color],[1,surface_color]],
                   showscale=False, name='Integrand', showlegend=True, opacity=0.5),
        go.Mesh3d(x=vx, y=vy, z=vz, i=i_l, j=j_l, k=k_l,
                  color='#1f77b4', opacity=block_opacity, name='Blocks'),
        go.Scatter3d(x=x_pts, y=y_pts, z=z_pts,
                     mode='markers', marker=dict(color='black', size=5),
                     name='Gauss points'),
        go.Scatter3d(x=ex, y=ey, z=ez,
                     mode='lines', line=dict(color='black', width=1),
                     name='Edges', showlegend=False)
    ],
    frames=frames
)

# Slider
steps = []
for n in range(1, n_max+1):
    steps.append(dict(method='animate',
                      args=[[str(n)], dict(mode='immediate', frame=dict(duration=0, redraw=True), transition=dict(duration=0))],
                      label=str(n)))
sliders = [dict(active=init_n-1, pad={'t':50}, currentvalue={'prefix':'Number of Points: '}, steps=steps)]

fig.update_layout(
    # title=f'exact integral = {56/15:.4f}, n={init_n}',
    title=f'Example showing how the integral is approximated',
    scene=dict(xaxis_title='ξ', yaxis_title='η', zaxis_title='f(ξ,η)'),
    sliders=sliders,
    width=1000, height=800)

fig.show()
